# Query Optimization for Mere Humans

# About Me 🙈

- Data architect at bigabid 👷.
- Data architect consultant 🤓.
- Big passion for python, big data, databases and machine learning 🐍🤖.
- Online at [medium](https://medium.com/@Eyaltra) | [website](https://eyaltrabelsi.github.io/) 🌐

# Today ⌛


<br>

- Query execution overview.
- Let's explain Explain.
- 2 Fake Examples.
- Are there easier ways?


# Query Execution Flow 🪜



<br>


<img src="https://www.interdb.jp/pg/img/fig-3-01.png" width="400" height="600">


**We want to reduce mental capacity !!** - [but if you want to go the extra mile](https://www.interdb.jp/pg/pgsql03.html).

 # Identify flaws in queries is tough🩺



<br>

- Its require skill to optimize sql queries.
- Its require skill to write sql queries without error.
- **My opinion**:
    - No good enough profilers.
    - No good enough debuggers.
    - Most tool are theoretical and led by the academy. 

- We are stuck with **execution plans**.
- **Buzzword alert !!** we should democretized execution plans.

# Let's explain Explain 📜



<br>

- **explain**: show what the planner planned to do.
- **explain analyze**: what the planner plans to do, **executes the query** and also shows how it did it.



``` mysql
EXPLAIN [ ( option [, ...] ) ] statement
```


``` sql
- ANALYZE [ boolean ]
- VERBOSE [ boolean ]
- COSTS [ boolean ]
- SETTINGS [ boolean ]
- BUFFERS [ boolean ]
- WAL [ boolean ]
- TIMING [ boolean ]
- SUMMARY [ boolean ]
- FORMAT { TEXT | XML | JSON | YAML }

```



- **Pro Tip**💃: Similar across databases.
- **Pro Tip**💃: go over an execution plan at least once.

# Why shouldn't I always use explain analyze? 😵‍💫


- Destructive operations.
- When resources are scarce:
    - The query never finishes.
    - Monitoring production.
- When you don't need actual stats:
    - Checking index usage.
    - Checking partition scans.
    - Checking query compilation*.

# Explain Anatomy 🫀

``` postgresql
EXPLAIN ANALYZE
SELECT COUNT(*) FROM users WHERE twitter != '';
```


![](basic_execution_plan.png)

- It's longer than our query :(.
- Real-world execution plans are overwhelming.

![](basic_execution_plan_high_level_stats.png)

- Finding the best execution plan is NP-hard.
- We may not get the optimal execution plan.
- The planning time may be longer than the execution time. 

- planning longer than execution time is pretty common when fetching one row using index scan and one should use prepare statements


![](node_details_execution_plan.png)


- Inverse tree.
- **Many operations**: 'Seq Scan', 'Values Scan', 'Sample Scan', 'Function Scan', 'CTEScan', 'Index Scan', 'Bitmap Heap Scan', 'Bitmap Index Scan', 'Index Only Scan','Subquery Scan', 'Hash Join','Nested Loop', 'Merge Join', 'Hash', 'Gather', 'Gather Merge','Unique','Result', 'SetOp', 'GroupAggregate','Aggregate', 'HashAggregate', 'WindowAgg', 'Limit', 'Sort', 'materialize', 'LockRows', 'Append', 'Merge Append' etc.



- **Pro Tip**💃: Cheat on your homework with [explain glossary](https://www.pgmustard.com/docs/explain).
- **Pro Tip**💃: Focus on what matter.

![](node_metrics_execution_plan.png)


- **Startup Cost**: arbirary units that represent estimated time to return the first row (aggregated).
- **Total Cost**: arbirary units that represent estimated time to return all the rows (aggregated). 
- **Plan Rows**: the estimated number of produced rows.
- **Plan Width**: the estimated average size of rows in bytes.
- **Actual Startup Time**: the time it took to return the first row in milliseconds (aggregated). 
- **Actual Total Time**: the time it took to return all the rows in milliseconds (per-loop average and aggregated).
- **Actual Rows**: the actual number of produced rows (per-loop average).
- **Actual Loops**: the number of loops the same node was executed.

- **Pro Tip**💃: loops are confusing.

# When explain will work? 🤔

<br>

    + Implies reasons why a query was slow.
            - Missing indices.
            - Overused indices.
            - Missing partitions.
            - Unoptimized database configurations*.
            - Redundant Operations.
            - Stale statistics.
    + Implies reason why query cardinality is wrong:
            - Missing records.
            - Too many records.
            - Duplications. 

    - Tells you why a particular optimization is not used.
    - Explain how to rewrite your queries.

# Example: Performance Optimization  🐆

``` postgresql
EXPLAIN ANALYZE
SELECT COUNT(*) FROM users WHERE twitter != '';
```


![](basic_execution_before_index.png)


- We perform a [sequential scan](https://www.pgmustard.com/docs/explain/sequential-scan) on the users table.
- The scan filters out 2,487,813 rows.
- It takes us 1.27 seconds to do all of this.

![](google_search.png)

``` postgresql
CREATE INDEX twitter_test ON users (twitter)
```

![](basic_execution_with_index.png)

- We perform an [index only scan](https://www.pgmustard.com/docs/explain/index-only-scan) on the users table.
- It takes us 0.29 seconds instead of 1.27 seconds.
- We could use buffers to be more precise.

- **Pro Tip**💃: optimize queries a step by step.
- **Pro Tip**💃: When comparing execution plans, look at several metrics. 

``` postgresql
EXPLAIN (ANALYZE, BUFFERS)
SELECT COUNT(*) FROM users WHERE twitter != ''
```



![](basic_execution_with_index_with_buffers.png)

- We use 51,854 buffers (400 MB). 
- The scan filters out 2,487,813 rows.
- We can create a more concise index using partial indices.

- **Pro Tip**💃: there is no free lunch.

# Good optimization options 🤞🏻

- Picking the right scan method.
- Picking the right join method.
- Picking the right join order.
- Push Filters as soon as possible.
- Reducing disk IO operations when needed.    


- **Pro Tip**💃: can enable/disable settings to check optimizations.


# Example: Empty Results 🐛

``` postgresql
EXPLAIN ANALYZE
SELECT COUNT(*) FROM users WHERE twitter = 'd0n@ldtrump';
```


![](empty_execution_plan_broken.png)

- We perform a [sequential scan](https://www.pgmustard.com/docs/explain/sequential-scan) on the users table.
- The scan filters out all rows using a Filter.

![](twitter.png)

``` postgresql
EXPLAIN ANALYZE
SELECT COUNT(*) FROM users WHERE twitter = 'donaldtrump';
```


![](empty_execution_plan_fixed.png)

- **Pro Tip**💃: in case we know a problem exists it is a productive tool.
- **Pro Tip**💃: in case we don't know a problem exists it may protect us.


# Aren't there easier ways?! 🙏


- UI can be nice.
- Hints how to rewrite your queries. 
- Hints why a particular optimization is not used. 
- Hints why/where a particular issue orinated.


- Cool tools out there:
    - [pev2](https://github.com/dalibo/pev2)
    - [eversql](https://www.eversql.com/)
    - [pgMustard](https://www.pgmustard.com/)
    - [QueryFlow](https://github.com/eyaltrabelsi/query-flow)


- **Pro Tip**💃: use one of these to make life easy.


## [pev2](https://github.com/dalibo/pev2)


![](pev2.png)

    + Mature
    + UI indicates the proportions of metrics.
    + UI indicates 'problematic' operations.
    + Aimed for all database users.
    + Not opinionated.
    - Support only Postgresql.
    - Support only a single query.
    - Limited support on multiple metrics.


## [QueryFlow](https://github.com/eyaltrabelsi/query-flow)

<img src='queryflow_example.png'>

    + Support several engines.
    + Operations are linkable with examples.
    + UI indicates 'problematic' operations.
    + UI indicates the proportions of metrics.
    + Support multiple metrics.
    + Support multiple queries.
    + Aimed for all database users, including the vendors.
    - Not mature
    - Very opinionated.
    - Require installations*.

# Optimistic Future 🔮



- Easy and intuitive as opening files in python (googling).
- Interactive feeling with functionalities like zoom, search, and suggestions.
- Proactive // Reactive.

![](https://i.pinimg.com/originals/b9/0a/79/b90a79b4c361d079144597d0bcdd61de.jpg)

[video 1](https://www.youtube.com/watch?v=Ls-uE1V31lE&list=WL&index=5&ab_channel=PostgresConference)
[video 2](https://www.youtube.com/watch?v=mCwwFAl1pBU&ab_channel=SouthernCaliforniaLinuxExpo)